### Use ``Open Route Service API`` to retrieve distance matrix (Single Call version)
#### Quota 500 per day, approx 1 year of merged rental-place dataset per day. 
#### Specify the ``year`` of rental and place dataset before running 
#### The following processes 2022 separately since no place data is available for 2022, by assumption we use the most recent 2021 instead

In [1]:
import pandas as pd
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../../scripts/')
import openrouteservice as ors
from add_distance import add_distance_time

In [2]:
PLACE_COLS = ['Place_Names', 'latitude', 'longitude', 'year', 'place_type', 'SA2_CODE']
place_df = pd.read_csv('../../data/curated/features_of_interst/result_place_all_with_sa2.csv')[PLACE_COLS]
print(place_df.shape)

(75786, 6)


### Alter the place and property dataframes for specific year (2013, 2014, ... 2022)

In [3]:
place_df = place_df[place_df['year'] == 2021]
place_df = place_df.drop_duplicates(subset=['Place_Names', 'latitude']).reset_index(drop=True)
print(place_df.shape)
place_df.head(10)

(9283, 6)


,Place_Names,latitude,longitude,year,place_type,SA2_CODE
0,WEST MELBOURNE MARKET,-37.81040,144.92237,2021,market,206041127
1,LILYDALE MARKET,-37.76040,145.34931,2021,market,211051278
2,PRESTON MARKET,-37.73790,145.00264,2021,market,209021428
3,PRAHRAN MARKET,-37.84615,144.99235,2021,market,206061516
4,VICTORIA MARKET,-37.80651,144.95653,2021,market,206041504
5,SOUTH MELBOURNE MARKET,-37.83263,144.95681,2021,market,206051512
6,CROYDON MARKET,-37.79374,145.27820,2021,market,211031451
7,SWAN HILL HOSPITAL,-35.34040,143.55626,2021,hospital,215031404
8,CLUNES HOSPITAL,-37.30179,143.78265,2021,hospital,201021010
9,MALDON HOSPITAL,-36.99270,144.06400,2021,hospital,202021028


In [4]:
PROPERTY_COLS = ['address', 'latitude', 'longitude', 'nbed', 'nbath', 'ncar', 'weekly_rent', 'type', 'postcode', 'year', 'month', 'residence_type', 'SA2_CODE']
property_df = pd.read_csv('../../data/curated/property_all_with_SA2/2022_property_with_SA2.csv')[PROPERTY_COLS]
# 2022 example
print(property_df.shape)
property_df.head(10)

(73107, 13)


,address,latitude,longitude,nbed,nbath,ncar,weekly_rent,type,postcode,year,month,residence_type,SA2_CODE
0,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,3,House,202021027
1,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,2,House,202021027
2,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,Rental_residential,3683,2022,9,Other,204031070
3,"17 PARK STREET, CHILTERN",-36.148025,146.608721,3.0,2.0,0,350,Rental_residential,3683,2022,8,Other,204031070
4,"1/20 NICKLESS STREET, CHILTERN",-36.154388,146.603242,2.0,1.0,1,320,House,3683,2022,7,House,204031070
5,"1/20 NICKLESS STREET, CHILTERN",-36.154388,146.603242,2.0,1.0,1,320,House,3683,2022,5,House,204031070
6,"1/20 NICKLESS STREET, CHILTERN",-36.154388,146.603242,2.0,1.0,1,300,House,3683,2022,1,House,204031070
7,"40 EPSOM ROAD, CHILTERN",-36.151454,146.605909,3.0,1.0,0,370,House,3683,2022,6,House,204031070
8,"7 CONNESS STREET, CHILTERN",-36.151025,146.607179,2.0,1.0,0,350,House,3683,2022,5,House,204031070
9,"3/1 OXFORD STREET, CHILTERN",-36.150694,146.606488,2.0,1.0,1,300,none,3683,2022,4,Other,204031070


In [5]:
merged_df = property_df.merge(place_df, how='inner', on='SA2_CODE')
print(f"shape = {merged_df.shape}")
print(f"Distinct sa2 codes = {merged_df['SA2_CODE'].nunique()}")
merged_df = merged_df.rename(columns={'latitude_x': 'latitude_ori', 'longitude_x': 'longitude_ori', 'latitude_y': 'latitude_des', 'longitude_y': 'longitude_des', 'year_x': 'year'})
merged_df.drop('year_y', axis=1, inplace=True)
merged_df.head(15)

shape = (1351949, 18)
Distinct sa2 codes = 502


,address,latitude_ori,longitude_ori,nbed,nbath,ncar,weekly_rent,type,postcode,year,month,residence_type,SA2_CODE,Place_Names,latitude_des,longitude_des,place_type
0,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,3,House,202021027,MOUNT ALEXANDER HOSPITAL,-37.05068,144.22126,hospital
1,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,3,House,202021027,CASTLEMAINE COMMUNITY HOSPITAL,-37.05068,144.22126,hospital
2,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,3,House,202021027,CASTLEMAINE POLICE STATION,-37.06373,144.21937,police
3,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,3,House,202021027,Castlemaine High School,-37.05054,144.22750,secondary
4,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,3,House,202021027,CASTLEMAINE TECHNICAL COLLEGE,-37.06346,144.22098,secondary
5,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,3,House,202021027,Castlemaine Secondary College- Junior Campus,-37.05054,144.22750,secondary
6,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,3,House,202021027,Castlemaine Secondary College - Senior Campus,-37.07707,144.21453,secondary
7,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,3,House,202021027,OLIVET CHRISTIAN COLLEGE,-37.08848,144.20543,secondary
8,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,3,House,202021027,WINTERS FLAT PRIMARY SCHOOL,-37.07016,144.20749,primary
9,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,3,House,202021027,CASTLEMAINE NORTH PRIMARY SCHOOL,-37.05783,144.21899,primary


### Create the directory to store merged distance data

In [6]:
if not os.path.exists('../../data/featured'):
    os.makedirs('../../data/featured')

#### Specify the ``client`` and ``year`` here, as well as adding a mode of testing/saving

In [7]:
client = ors.Client('5b3ce3597851110001cf6248ce6c95ac96814219a4c3a7741f323b73') # Phikho-caz's api key 2500, back up for at least 4 full year

In [8]:
added_distance_merged_df = add_distance_time(merged_df, 2022, client, 'saving') # specify the year here, client id, and mode of running (testing or saving)
#added_distance_merged_df = add_distance_time(subset, 2013)

Executing SA2 Code 202021027
Subset size = 2074, Places count = 17, Property count = 122
To Place Distance Grand List, Normal Branch, length = 2074
Executing SA2 Code 204031070
Subset size = 170, Places count = 10, Property count = 17
To Place Distance Grand List, Normal Branch, length = 2244
Executing SA2 Code 210021235
Subset size = 1079, Places count = 13, Property count = 83
To Place Distance Grand List, Normal Branch, length = 3323
Executing SA2 Code 211051275
Subset size = 186, Places count = 3, Property count = 62
To Place Distance Grand List, Normal Branch, length = 3509
Executing SA2 Code 211051278
Subset size = 3640, Places count = 20, Property count = 182
To Place Distance Grand List, Slice Branch, length = 5329
To Place Distance Grand List, Slice Branch, length = 7149
Executing SA2 Code 211051276
Subset size = 2325, Places count = 25, Property count = 93
To Place Distance Grand List, Normal Branch, length = 9474
Executing SA2 Code 205041094
Subset size = 4284, Places count 

In [ ]:
added_distance_merged_df.head(20)

,address,latitude_ori,longitude_ori,nbed,nbath,ncar,weekly_rent,type,postcode,year,...,longitude_des,place_type,dist_to_place_M,dist_to_place_KM,time_to_place_S,time_to_place_MIN,dist_to_cbd_M,dist_to_cbd_KM,time_to_cbd_S,time_to_cbd_MIN
0,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,...,144.22750,secondary,7895.97,7.89597,601.73,10.028833,116076.56,116.07656,5236.62,87.277
1,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,...,144.22098,secondary,5254.10,5.25410,441.94,7.365667,116076.56,116.07656,5236.62,87.277
2,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,...,144.22750,secondary,7895.97,7.89597,601.73,10.028833,116076.56,116.07656,5236.62,87.277
3,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,...,144.21453,secondary,6224.54,6.22454,625.11,10.418500,116076.56,116.07656,5236.62,87.277
4,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,...,144.20543,secondary,8087.73,8.08773,622.79,10.379833,116076.56,116.07656,5236.62,87.277
5,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,...,144.20749,primary,6406.50,6.40650,619.35,10.322500,116076.56,116.07656,5236.62,87.277
6,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,...,144.21899,primary,6107.61,6.10761,423.85,7.064167,116076.56,116.07656,5236.62,87.277
7,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,...,144.22233,primary,4592.83,4.59283,320.81,5.346833,116076.56,116.07656,5236.62,87.277
8,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,...,144.26116,primary,673.75,0.67375,57.82,0.963667,116076.56,116.07656,5236.62,87.277
9,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,...,144.20466,primary,8474.68,8.47468,646.31,10.771833,116076.56,116.07656,5236.62,87.277
